In [30]:
#Spell Checker using Hamming distance


In [31]:
%pylab inline
import re
import math
import string
import collections
from collections import Counter
from __future__ import division
import csv
import pandas as pd
from collections import Counter
from itertools import product
import functools

Populating the interactive namespace from numpy and matplotlib


/Users/joby/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['product']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [32]:
#File_Corpus = 'test_data/count_1w.txt'

File_Corpus = 'test_data/big.txt'
#File_Corpus = 'test_data/sherlockholmes.txt'
File_Spell_Errors='spell-errors.txt'

Test_Data = 'test_data/test.csv'

#Initialize global variables
VERBOSE = True
vowels = set('aeiouy')
alphabet = set('abcdefghijklmnopqrstuvwxyz')

def words(text): 
    return re.findall(r'\w+', text.lower())


def train(text, model=None):
    """generate or update a word model (dictionary of word:frequency)"""
    model = collections.defaultdict(lambda: 0) if model is None else model
    for word in words(text):
        model[word] += 1
    return model

word_model = train(open(File_Corpus).read())
WORDS = set(word_model)

    
#WORDS = Counter(words(open(File_Corpus).read()))

print('Total Word Set: ', len(WORDS))


def read_csv_file(filename):
    error_words = []
 
    with open(filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        next(csvReader)
        for row in csvReader:
            error_words.append(row[1])
 
    return error_words
    
    
def test_corpus(filename):
    text = open(filename).read()
    return dict((x.split(' ')[0],int(x.split(' ')[1])) for x in text.split('\n'))

#WORDS = test_corpus(File_Corpus)

test_words = read_csv_file(Test_Data)

def write_to_file(wordlist):
    df = pd.DataFrame(wordlist)
    df.to_csv("Output/test_hamming_submit.csv")

    

Total Word Set:  8166


In [33]:
def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(list(counter.values()))
    return lambda x: counter[x]/N

#P = pdist(WORDS)


In [34]:
#Utility functions

def numberofdupes(string, idx):
    """return the number of times in a row the letter at index idx is duplicated"""
    # "abccdefgh", 2  returns 1
    initial_idx = idx
    last = string[idx]
    while idx+1 < len(string) and string[idx+1] == last:
        idx += 1
    return idx-initial_idx

def hamming_distance(word1, word2):
    if word1 == word2:
        return 0
    dist = sum(list(map(str.__ne__, word1[:len(word2)], word2[:len(word1)])))
    dist = max([word2, word1]) if not dist else dist+abs(len(word2)-len(word1))
    return dist

def frequency(word, word_model):
    return word_model.get(word, 0)

def translate_score(score):   
    if re.match("^[A-Za-z_-]*$", str(score)):
        return 10000;
    else:
        return score


In [35]:
def correction(word): 
    "Most probable spelling correction for word."
    #print(candidates(word))
    try:
        return max(candidates(word), key=P)
    except Exception as e:
        return 'NULL'

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS.keys())

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [36]:
### POSSIBILITIES ANALYSIS

def variants(word):
    """get all possible variants for a word"""
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [a + b[1:] for a, b in splits if b]
    transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
    replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
    inserts    = [a + c + b for a, b in splits for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def double_variants(word):
    """get variants for the variants for a word"""
    return set(s for w in variants(word) for s in variants(w))

def reductions(word):
    """return flat option list of all possible variations of the word by removing duplicate letters"""
    word = list(word)
    # ['h','i', 'i', 'i'] becomes ['h', ['i', 'ii', 'iii']]
    for idx, l in enumerate(word):
        n = numberofdupes(word, idx)
        # if letter appears more than once in a row
        if n:
            # generate a flat list of options ('hhh' becomes ['h','hh','hhh'])
            flat_dupes = [l*(r+1) for r in range(n+1)][:3] # only take up to 3, there are no 4 letter repetitions in english
            # remove duplicate letters in original word
            for _ in range(n):
                word.pop(idx+1)
            # replace original letter with flat list
            word[idx] = flat_dupes

    # ['h',['i','ii','iii']] becomes 'hi','hii','hiii'
    for p in product(*word):
        yield ''.join(p)

def vowelswaps(word):
    """return flat option list of all possible variations of the word by swapping vowels"""
    word = list(word)
    # ['h','i'] becomes ['h', ['a', 'e', 'i', 'o', 'u', 'y']]
    for idx, l in enumerate(word):
        if type(l) == list:
            pass                        # dont mess with the reductions
        elif l in vowels:
            word[idx] = list(vowels)    # if l is a vowel, replace with all possible vowels

    # ['h',['i','ii','iii']] becomes 'hi','hii','hiii'
    for p in product(*word):
        yield ''.join(p)

def both(word):
    """permute all combinations of reductions and vowelswaps"""
    for reduction in reductions(word):
        for variant in vowelswaps(reduction):
            yield variant

In [37]:
#Generate best suggestion list

def suggestions(word, real_words, short_circuit=True):
    """get best spelling suggestion for word
    return on first match if short_circuit is true, otherwise collect all possible suggestions"""
    word = word.lower()
    if short_circuit:   # setting short_circuit makes the spellchecker much faster, but less accurate in some cases
        return ({word}                      & real_words or   #  caps     "inSIDE" => "inside"
                set(reductions(word))       & real_words or   #  repeats  "jjoobbb" => "job"
                set(vowelswaps(word))       & real_words or   #  vowels   "weke" => "wake"
                set(variants(word))         & real_words or   #  other    "nonster" => "monster"
                set(both(word))             & real_words or   #  both     "CUNsperrICY" => "conspiracy"
                set(double_variants(word))  & real_words or   #  other    "nmnster" => "manster"
                {"NO SUGGESTION"})
    else:
        return ({word}                      & real_words or
                (set(reductions(word))  | set(vowelswaps(word)) | set(variants(word)) | set(both(word)) | set(double_variants(word))) & real_words or
                {"NO SUGGESTION"})

       
    
   

In [38]:
# Find the suggestions:

def best(inputted_word, suggestions, word_model=None):
    """choose the best suggestion in a list based on lowest hamming distance from original word, or based on frequency if word_model is provided"""

    suggestions = list(suggestions)

    def comparehamm(one, two):
        score1 = hamming_distance(inputted_word, one)
        score1 = translate_score(score1)
        score2 = hamming_distance(inputted_word, two)
        score2 = translate_score(score2)
        return (score1 > score2)-(score1 < score2)  # lower is better

    def comparefreq(one, two):
        score1 = frequency(one, word_model)
        score2 = frequency(two, word_model)
        return (score2>score1)-(score2<score1)
        
    freq_sorted = sorted(suggestions, key=functools.cmp_to_key(comparefreq))[:1]     # take the top 10
    print ('FREQ', freq_sorted)
    hamming_sorted = sorted(suggestions, key=functools.cmp_to_key(comparehamm))[:1]  # take the top 10
    print ('HAM', hamming_sorted)
    return hamming_sorted[0]
    #return ''


In [39]:
#P('imidatly')*math.pow(10,10)

word= 'nessisitates'

possibilities = suggestions(word, WORDS, short_circuit=False)
short_circuit_result = suggestions(word, WORDS, short_circuit=True)
#print(possibilities)

print(correction('nessisitates'))


NULL


In [40]:

#if VERBOSE:
 #   best(word, possibilities, word_model)
  #  print ('---')
  #  print [(x, WORDS[x]) for x in short_circuit_result]

if VERBOSE:
    best(word, short_circuit_result, word_model)

FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']


In [41]:

correct_list=[]
for word in test_words:
    short_circuit_result = suggestions(word, WORDS, short_circuit=False)
   # print (w)
    correct_list.append(best(word, short_circuit_result, word_model))

#for w in correct_list:
 #   print(w)

FREQ ['consented']
HAM ['consented']
FREQ ['beginning']
HAM ['beginnings']
FREQ ['problem']

/Users/joby/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.
  del sys.path[0]



HAM ['problems']
FREQ ['given']
HAM ['drives']
FREQ ['eustace']
HAM ['eustace']
FREQ ['quick']
HAM ['gush']
FREQ ['lonely']
HAM ['lonely']
FREQ ['company']
HAM ['compass']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['independent']
HAM ['independent']
FREQ ['orange']
HAM ['ragged']
FREQ ['party']
HAM ['poetry']
FREQ ['leave']
HAM ['devil']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['triangular']
HAM ['triangular']
FREQ ['unexpected']
HAM ['unexpected']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['valuable']
HAM ['valuable']
FREQ ['other']
HAM ['fuller']
FREQ ['loitering']
HAM ['loitering']
FREQ ['visits']
HAM ['visits']
FREQ ['amiable']
HAM ['avail']
FREQ ['separate']
HAM ['temperate']
FREQ ['necessary']
HAM ['necessary']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['receive']
HAM ['regent']
FREQ ['mine']
HAM ['retire']
FREQ ['initials']
HAM ['initials']
FREQ ['magnific

FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['building']
HAM ['buildings']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['beginning']
HAM ['beginnings']
FREQ ['and']
HAM ['an']
FREQ ['moment']
HAM ['moments']
FREQ ['the']
HAM ['then']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['questionable']
HAM ['questionable']
FREQ ['envelope']
HAM ['develop']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['appointment']
HAM ['appointment']
FREQ ['visible']
HAM ['variable']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['equally']
HAM ['equal']
FREQ ['boots']
HAM ['bowls']
FREQ ['demand']
HAM ['demand']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['recently']
HAM ['recently']
FREQ ['clear']
HAM ['clerks']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['feeling']
HAM ['feelings']
FREQ ['family']
HAM ['familiar']
FREQ ['acted']
HAM ['acted']
FREQ ['accepting']
HAM ['accepting'

FREQ ['discretion']
HAM ['discretion']
FREQ ['hand']
HAM ['hands']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['person']
HAM ['persons']
FREQ ['senior']
HAM ['senior']
FREQ ['apologise']
HAM ['apologise']
FREQ ['separate']
HAM ['separate']
FREQ ['me']
HAM ['then']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['buy']
HAM ['brisk']
FREQ ['easy']
HAM ['easily']
FREQ ['coat']
HAM ['coat']
FREQ ['seem']
HAM ['seem']
FREQ ['seeing']
HAM ['peering']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['difficult']
HAM ['difficulty']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['case']
HAM ['taste']
FREQ ['remained']
HAM ['regained']
FREQ ['control']
HAM ['control']
FREQ ['offered']
HAM ['suffered']
FREQ ['NO SUGGESTION']
HAM ['NO SUGGESTION']
FREQ ['through']
HAM ['though']
FREQ ['years']
HAM ['bears

In [42]:
#Saving to file

write_to_file(correct_list)